# CSV → Normalized Dataset → Parquet (Athena-ready)

This notebook documents the final preprocessing pipeline used to prepare a support ticket dataset
for querying in Amazon Athena.

During development, several attempts were made to make the original CSV compatible with Athena
(including cleaning multiline fields and testing different parsing approaches), but these attempts
remained unreliable. The final, robust solution was to convert the dataset to Parquet after
performing data normalization.

## Manual preprocessing (outside Python)

Before loading the dataset into the pipeline, the raw CSV was manually reviewed and lightly cleaned in Excel to:

- Remove non-English rows
- Drop irrelevant columns
- Rename columns for clarity
- Reduce noise in free-text fields

This ensured the dataset was relevant and analyzable before applying automated transformations in Python.

In [2]:
# CONFIG
from pathlib import Path
import pandas as pd

# --- USER CONFIG ---
INPUT_FILE = "data/raw/tickets.csv"     # change this for any new file
OUTPUT_NAME = "output.parquet"

# Path
PROJECT_ROOT = Path.cwd()

# If we are inside /notebooks, go up one level
if not (PROJECT_ROOT / "data").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

INPUT_PATH = PROJECT_ROOT / INPUT_FILE
OUTPUT_DIR = PROJECT_ROOT / "data" / "processed"
OUTPUT_PATH = OUTPUT_DIR / OUTPUT_NAME

print("Project root:", PROJECT_ROOT)
print("Input path:", INPUT_PATH)
print("Exists:", INPUT_PATH.exists())
print("Output will be:", OUTPUT_PATH)

Project root: /Users/mohamadarnaout/Documents/GitHub/enterprise-ai-ops-assistant
Input path: /Users/mohamadarnaout/Documents/GitHub/enterprise-ai-ops-assistant/data/raw/tickets.csv
Exists: True
Output will be: /Users/mohamadarnaout/Documents/GitHub/enterprise-ai-ops-assistant/data/processed/output.parquet


## Load Raw Data

The cleaned CSV is loaded using pandas, which correctly handles complex quoting and multiline fields.
All values are loaded as strings to preserve consistency and avoid implicit type issues.

In [3]:
df = pd.read_csv(INPUT_PATH, encoding="utf-8-sig", dtype=str)
df.head(10)

,ticket_id,ticket_type,priority,category,assigned_to,description
0,1,Request,high,IT Support,IT Services,"Dear IT Services Support Team, \n\nI am reachi..."
1,2,Incident,high,Technical Support,IT Services,"Dear Customer Support,\n\nI am experiencing a ..."
2,3,Change,high,Service Outages and Maintenance,IT Services,"Dear IT Services Customer Support,\n\nI am wri..."
3,4,Incident,medium,Product Support,Tech Online Store,"Dear Tech Online Store Support Team,\n\nI hope..."
4,5,Request,medium,Technical Support,IT Services,"Dear Customer Support,\n\nI hope this message ..."
5,6,Request,medium,Technical Support,IT Services,"Dear IT Services Customer Support,\n\nI hope t..."
6,7,Incident,high,Service Outages and Maintenance,IT Services,"Dear Customer Support,\n\nWe are currently exp..."
7,8,Request,high,Billing and Payments,IT Services,"Dear Customer Support, \n\nI hope this message..."
8,9,Problem,low,Technical Support,Tech Online Store,"Dear Tech Online Store Customer Support, I am ..."
9,10,Problem,medium,Technical Support,IT Services,"Dear Customer Care,\n\nI am encountering chall..."


## Validate Raw Data

Before applying transformations, we validate the structure and contents of the dataset by inspecting:

- Shape (rows, columns)  
- Column names  
- Data types  
- Missing values  
- Sample values per column  

This helps surface schema issues and unexpected values early.

In [12]:
print("Shape:", df.shape)
print("\nColumns:")
print(df.columns.tolist())

print("\nInfo:")
df.info()

print("\nMissing values per column:")
print(df.isnull().sum())

print("\nSample rows:")
display(df.head())

print("\nSample unique values per column (first 10):")
for col in df.columns:
    print(f"\n{col}")
    print(df[col].dropna().unique()[:10])

Shape: (501, 6)

Columns:
['ticket_id', 'ticket_type', 'priority', 'category', 'assigned_to', 'description']

Info:
<class 'pandas.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   ticket_id    501 non-null    str  
 1   ticket_type  501 non-null    str  
 2   priority     501 non-null    str  
 3   category     501 non-null    str  
 4   assigned_to  501 non-null    str  
 5   description  501 non-null    str  
dtypes: str(6)
memory usage: 389.3 KB

Missing values per column:
ticket_id      0
ticket_type    0
priority       0
category       0
assigned_to    0
description    0
dtype: int64

Sample rows:


,ticket_id,ticket_type,priority,category,assigned_to,description
0,1,Request,high,IT Support,IT Services,"Dear IT Services Support Team, \n\nI am reachi..."
1,2,Incident,high,Technical Support,IT Services,"Dear Customer Support,\n\nI am experiencing a ..."
2,3,Change,high,Service Outages and Maintenance,IT Services,"Dear IT Services Customer Support,\n\nI am wri..."
3,4,Incident,medium,Product Support,Tech Online Store,"Dear Tech Online Store Support Team,\n\nI hope..."
4,5,Request,medium,Technical Support,IT Services,"Dear Customer Support,\n\nI hope this message ..."



Sample unique values per column (first 10):

ticket_id
<ArrowStringArray>
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
Length: 10, dtype: str

ticket_type
<ArrowStringArray>
['Request', 'Incident', 'Change', 'Problem']
Length: 4, dtype: str

priority
<ArrowStringArray>
['high', 'medium', 'low']
Length: 3, dtype: str

category
<ArrowStringArray>
[                     'IT Support',               'Technical Support',
 'Service Outages and Maintenance',                 'Product Support',
            'Billing and Payments',                'Customer Service',
           'Returns and Exchanges',             'Sales and Pre-Sales',
                 'General Inquiry',                 'Human Resources']
Length: 10, dtype: str

assigned_to
<ArrowStringArray>
[                 'IT Services',            'Tech Online Store',
           'IT Consulting Firm', 'Software Development Company',
                 'Online Store',         'Adobe Photoshop 2024']
Length: 6, dtype: str

description
<Arro

## Cleaning & Normalization

Rather than attempting to force CSV compatibility, the focus of this step is data quality improvement.

The goal is to improve consistency across key categorical fields to ensure reliable grouping, filtering,
and aggregation in downstream analysis.

## Data Normalization (Python)

The following lightweight transformations are applied:

- Trim leading/trailing whitespace  
- Normalize casing for categorical consistency  
- Preserve semantics while reducing formatting noise  

These steps improve data quality without altering meaning.

In [19]:
df_clean = df.copy()

df_clean["ticket_type"] = df_clean["ticket_type"].str.strip().str.title()
df_clean["priority"] = df_clean["priority"].str.strip().str.lower()
df_clean["category"] = df_clean["category"].str.strip()
df_clean["assigned_to"] = df_clean["assigned_to"].str.strip()

## Export Final Dataset (Parquet)

The cleaned dataset is exported as Parquet, which provides:

- Reliable schema handling
- Efficient columnar storage
- Better compatibility with Athena / Glue
- Faster queries and lower cost

In [14]:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

df_clean.to_parquet(OUTPUT_PATH, index=False)

print(f"Parquet written to: {OUTPUT_PATH}")


Parquet written to: /Users/mohamadarnaout/Documents/GitHub/enterprise-ai-ops-assistant/data/processed/output.parquet


## Validate Parquet Output

We reload the Parquet file to confirm:
- Schema consistency
- Row counts match expectations
- File is readable for downstream systems

In [16]:
df_parquet = pd.read_parquet(OUTPUT_PATH)

print("Parquet shape:", df_parquet.shape)

assert df_parquet.shape == df_clean.shape, "Parquet validation failed!"

display(df_parquet.head())


Parquet shape: (501, 6)


,ticket_id,ticket_type,priority,category,assigned_to,description
0,1,Request,high,IT Support,IT Services,"Dear IT Services Support Team, \n\nI am reachi..."
1,2,Incident,high,Technical Support,IT Services,"Dear Customer Support,\n\nI am experiencing a ..."
2,3,Change,high,Service Outages and Maintenance,IT Services,"Dear IT Services Customer Support,\n\nI am wri..."
3,4,Incident,medium,Product Support,Tech Online Store,"Dear Tech Online Store Support Team,\n\nI hope..."
4,5,Request,medium,Technical Support,IT Services,"Dear Customer Support,\n\nI hope this message ..."


## Appendix: Attempted CSV Multiline Cleanup

An earlier experiment attempted to make the original CSV compatible with Athena by normalizing newline characters in multiline text fields (such as ticket descriptions).

The initial issue with the raw CSV was that Athena returned an incorrect row count (4365 instead of the expected 501), caused by multiline values breaking the structure of the file.

After applying a Python script to normalize newline characters, the row count issue was resolved and the cleaned CSV could be queried successfully in Athena. However, further validation showed that query results were still inconsistent and some fields appeared misaligned in certain cases.

This meant that although the CSV could be made to "work" syntactically, it did not reliably preserve data integrity.

The Parquet-based pipeline produced consistent, correct results across all validation queries and was therefore selected as the final format for analysis.




In [6]:
df_experiment = df.copy()

df_experiment = df_experiment.map(
    lambda x: x.replace("\r\n", " ").replace("\n", " ").replace("\r", " ")
    if isinstance(x, str) else x
)

changed = (df != df_experiment).any(axis=1).sum()
print(f"Rows modified by newline cleanup: {changed} / {len(df)}")


Rows modified by newline cleanup: 382 / 501


In [7]:
# Export experiment-cleaned CSV for Athena testing
EXPERIMENT_CSV_PATH = OUTPUT_DIR / "tickets_experiment_cleaned.csv"

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

df_experiment.to_csv(
    EXPERIMENT_CSV_PATH,
    index=False,
    encoding="utf-8",
    lineterminator="\n"
)

print(f"Experiment CSV written to: {EXPERIMENT_CSV_PATH}")


Experiment CSV written to: /Users/mohamadarnaout/Documents/GitHub/enterprise-ai-ops-assistant/data/processed/tickets_experiment_cleaned.csv
